#property indicator_chart_window
#property indicator_buffers 1
#property indicator_plots   1

#property indicator_label1  "Developing POC"
#property indicator_type1   DRAW_LINE
#property indicator_color1  clrOrange
#property indicator_width1  2

// ─────────────────────────────────────
input int    LookbackBars  = 200;
input double PriceStepMult = 1.0;

double PocLine[];

// ─────────────────────────────────────
int OnInit()
{
   SetIndexBuffer(0, PocLine, INDICATOR_DATA);
   ArraySetAsSeries(PocLine, true);
   return INIT_SUCCEEDED;
}

// ─────────────────────────────────────
int OnCalculate(
   const int rates_total,
   const int prev_calculated,
   const datetime& time[],
   const double& open[],
   const double& high[],
   const double& low[],
   const double& close[],
   const long& tick_volume[],
   const long& volume[],
   const int& spread[]
)
{
   if(rates_total < LookbackBars)
      return 0;

   const double step = _Point * PriceStepMult;
   int start = (prev_calculated > 0) ? prev_calculated - 1 : LookbackBars;

   for(int i = start; i < rates_total; i++)
   {
      int from = i;
      int to   = MathMin(i + LookbackBars - 1, rates_total - 1);

      double pLow  = low[from];
      double pHigh = high[from];

      for(int b = from; b <= to; b++)
      {
         if(low[b]  < pLow)  pLow  = low[b];
         if(high[b] > pHigh) pHigh = high[b];
      }

      int levels = int((pHigh - pLow) / step) + 1;
      if(levels <= 0 || levels > 3000)
      {
         PocLine[i] = EMPTY_VALUE;
         continue;
      }

      static double vol[];
      ArrayResize(vol, levels);
      ArrayInitialize(vol, 0.0);

      for(int b = from; b <= to; b++)
      {
         int idx = int((close[b] - pLow) / step);
         if(idx >= 0 && idx < levels)
            vol[idx] += (double)volume[b];
      }

      double maxVol = 0.0;
      int pocIdx = -1;

      for(int v = 0; v < levels; v++)
      {
         if(vol[v] > maxVol)
         {
            maxVol = vol[v];
            pocIdx = v;
         }
      }

      PocLine[i] = (pocIdx >= 0) ? pLow + pocIdx * step : EMPTY_VALUE;
   }

   return rates_total;
}
